# Predicting Human Gait Cycles Using Accelerometer Data

## Background

The human gait cycle is a very complex process which is defined as the period of time from the initial contact of one foot to the following occurrence of hte same event with the same foot.
[1] ('<scrip> src="https://dx.doi.org/10.1097%2F00008526-199710000-00004"></scrip>')

One gait segment can be simplied classified into the stance phase and the swing phase, or into 4, 5, 6, or even 8 phases. In this project, we used the 4-phase model - heel strike, flat foot, heel off, and swing.



## Libraries

In [1]:
import numpy as np
import pandas as pd

## Data Collection

The accelerometer on an iPhone was used to collect accleration data. Accelerometers tend to be the most popular data source for gait partitioning problems becuase it's durable, readily available and inexpensive. 

An accelerometer is an electromechanical device used to measure acceleration forces. Such forces may be static, like the continuous force of gravity or, as is the case with many mobile devices, dynamic to sense movement or vibrations.


https://www.livescience.com/40102-accelerometers.html

### Data Augmentation

Due to lack of time and participants, the training data came from 2 subjects with 10000 steps each. And the development and test data came from 2 other subjects with 100 steps each. Phone with the accelerometer was put at the back pocket of each subject.

## Data Preprocessing

## Visualization

## Modeling

3 models in parrallel

### Model 1 - HMM

6 phases


### Model 2 - CNN

multilayer perceptron

53

moving average technique


### Model 3 - 
Regularization

## Limitations

- Normal walking in the sagittal plane on level ground

## Future Work

- Increase the dataset size

- Better data collection
    - Gyroscope

    - Sensors on the shank, thigh and foot

    - foot angular velocity data
    
- Test on both healthy individuals and patients, different age groups

## References


Gait Partitioning Methods: A Systematic Review 
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4732099/
        